# OpenEO endpoint in Examind - Example

By Quentin BIALOTA (Geomatys)

Contact : quentin.bialota@geomatys.com

---

### **/!\ WARNING**
This initial implementation of openEO is not yet complete and may contain bugs. Please let me know if you find any. 

For the moment we are not compatible with openEO clients. In this tutorial we will use http requests to communicate with the examind openEO implementation.

---

**Prerequisites :**
- Have an examind (via docker) running on your machine
- Have imported sentinel-2 data in Examind (through the web ui http://localhost/8080/examind [login: admin / password: admin])

In [25]:
# Imports (Execute this cell once)
import requests
import json

### 1 - List openEO available processes

The first step we can do is to request the list of processes available via examind's openEO endpoint.

In [24]:
URL = "http://localhost:8080/examind/WS/openeo/process/WPS/processes"

r = requests.get(url = URL)

data = r.json()

# If you want to show the raw json output
# print(json.dumps(data, indent=2))

# Iterate over each process in the data
for process in data['processes']:
    print(f"ID: {process['id']}")

    print(f"Description: {process['description']}")

    print("Parameters:")
    for parameter in process['parameters']:
        print(f"- {parameter['name']} {parameter['schema']['type']}")
        print(f"  Description: {parameter['description']}")
        if 'subtype' in parameter['schema'] and parameter['schema']['subtype'] is not None:
            print(f"  Subtype: {parameter['schema']['subtype']}")

    print("Returns:")
    if process['returns']['schema'] is not None:
        print(f"  Description: {process['returns']['description']}")
        print(f"  Type: {process['returns']['schema']['type']}")
        if 'subtype' in process['returns']['schema'] and process['returns']['schema']['subtype'] is not None:
            print(f"  Subtype: {process['returns']['schema']['subtype']}")

    print("\n")

ID: examind.coverage:math:sumWithValue
Description: Sum a Coverage with a single value
Parameters:
- coverage ['OBJECT']
  Description: Input coverage
  Subtype: org.apache.sis.coverage.grid.GridCoverage
- value ['NUMBER']
  Description: Input value
  Subtype: Real
Returns:
  Description: Result as a coverage
  Type: ['OBJECT']


ID: examind.sensor.correct.fieldname
Description: Fill the sensor fields label.
Parameters:
- observation_provider_id ['STRING']
  Description: Observation provider identifier.
  Subtype: CharacterString
- sensor_id ['STRING']
  Description: Sensor identifier (Optional).
  Subtype: CharacterString
Returns:


ID: examind.test.multiple.type
Description: Test multiple types.
Parameters:
- double.array.input ['ARRAY']
  Description: Double array input.
  Subtype: Real
- double.prim.array.input ['ARRAY']
  Description: Double primitive array input.
  Subtype: Real
- double.multiple.input ['NUMBER']
  Description: Double multiple input.
  Subtype: Real
- integer.arr

### 2 - Create your own process

This step should eventually work via openEO web Editor (https://github.com/Open-EO/openeo-web-editor), but for the moment our implementation is not compatible (nor is it compatible with openEO clients) because some of the paths to the APIs are not standard (the way the API works, however, is compliant).

For the moment, we're going to create an EVI (Enhanced Vegetation Index) process by ourselves, directly via an http request !

In [38]:
# Set Json of my EVI-Sentinel process

data = {
    'id': 'evi-sentinel',
    'summary': 'Enhanced Vegetation Index',
    'description': 'Computes the Enhanced Vegetation Index (EVI). It is computed with the following formula: `2.5 * (NIR - RED) / (1 + NIR + 6*RED + 7.5*BLUE)`.',
    'parameters': [
        {
            'name': 'dataId',
            'description': 'Sentinel-2 data id',
            'schema': {
                'type': 'String'
            }
        },
        {
            'name': 'bbox',
            'description': 'Spatial extent',
            'schema': {
                'type': 'Object'
            }
        },
        {
            'name': 'temporal',
            'description': 'Temporal extent',
            'schema': {
                'type': 'Array',
                'subtype': 'String'
            }
        },
        {
            'name': 'bands',
            'description': 'Bands to load',
            'schema': {
                'type': 'Array',
                'subtype': 'String'
            }
        }
    ],
    'returns': {
        'description': 'Computed EVI.',
        'schema': {
            'type': 'object'
        }
    },
    'process_graph': {
        'load': {
            'process_id': 'examind.coverage.load',
            'arguments': {
                'coverageId': {
                    'from_parameter': 'dataId'
                },
                'spatial_extent': {
                    'from_parameter': 'bbox'
                },
                'temporal_extent': {
                    'from_parameter': 'temporal'
                },
                'bands': {
                    'from_parameter': 'bands'
                }
            }
        },
        'bs3': {
            'process_id': 'geotoolkit.coverage:bandselect',
            'arguments': {
                'coverage': {
                    'from_node': 'load'
                },
                'bands': [
                    1
                ]
            }
        },
        'bs4': {
            'process_id': 'geotoolkit.coverage:bandselect',
            'arguments': {
                'coverage': {
                    'from_node': 'load'
                },
                'bands': [
                    2
                ]
            }
        },
        'bs8': {
            'process_id': 'geotoolkit.coverage:bandselect',
            'arguments': {
                'coverage': {
                    'from_node': 'load'
                },
                'bands': [
                    3
                ]
            }
        },
        'sub': {
            'process_id': 'examind.coverage:math:substract',
            'arguments': {
                'first': {
                    'from_node': 'bs8'
                },
                'second': {
                    'from_node': 'bs3'
                }
            }
        },
        'p1': {
            'process_id': 'examind.coverage:math:multiplyWithValue',
            'arguments': {
                'value': 6.0,
                'coverage': {
                    'from_node': 'bs3'
                }
            }
        },
        'p2': {
            'process_id': 'examind.coverage:math:multiplyWithValue',
            'arguments': {
                'value': 7.5,
                'coverage': {
                    'from_node': 'bs4'
                }
            }
        },
        'sumCoverages1': {
            'process_id': 'examind.coverage:math:sum',
            'arguments': {
                'first': {
                    'from_node': 'bs8'
                },
                'second': {
                    'from_node': 'p1'
                }
            }
        },
        'sumCoverages2': {
            'process_id': 'examind.coverage:math:sum',
            'arguments': {
                'first': {
                    'from_node': 'sumCoverages1'
                },
                'second': {
                    'from_node': 'p2'
                }
            }
        },
        'sum': {
            'process_id': 'examind.coverage:math:sumWithValue',
            'arguments': {
                'coverage': {
                    'from_node': 'sumCoverages2'
                },
                'value': 1.0
            }
        },
        'div': {
            'process_id': 'examind.coverage:math:divide',
            'arguments': {
                'first': {
                    'from_node': 'sub'
                },
                'second': {
                    'from_node': 'sum'
                }
            }
        },
        'p3': {
            'process_id': 'examind.coverage:math:multiplyWithValue',
            'arguments': {
                'value': 2.5,
                'coverage': {
                    'from_node': 'div'
                }
            }
        },
        'save': {
            'process_id': 'examind.coverage.save_result',
            'arguments': {
                'format': 'GTIFF',
                'coverage': {
                    'from_node': 'p3'
                }
            },
            'result': True
        }
    }
}

You can check if your process is valid with `/validation` endpoint

In [39]:
URL = "http://localhost:8080/examind/WS/openeo/process/WPS/validation"

headers = {'Content-Type': 'application/json', 'Accept': '*/*'}

r = requests.post(url=URL, data=json.dumps(data), headers=headers)

print(f"Status {r.status_code}")
print(json.dumps(r.json(), indent=2))

Status 200
{
  "errors": [
    {
      "id": "360697e8-27a0-453d-951d-5b437cba3af3",
      "code": "ProcessIDAlreadyExist",
      "message": "The process id specified already exist, you cannot use the same to add this process.",
      "links": []
    }
  ]
}


In [42]:
URL = "http://localhost:8080/examind/WS/openeo/process/WPS/process_graphs/evi-sentinel"

headers = {'Content-Type': 'application/json', 'Accept': '*/*'}

r = requests.put(url=URL, data=json.dumps(data), headers=headers)

print(f"Status {r.status_code}")
print(f"Data: {r.text}")

Status 500
Data: <!doctype html><html lang="en"><head><title>HTTP Status 500 – Internal Server Error</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 500 – Internal Server Error</h1><hr class="line" /><p><b>Type</b> Status Report</p><p><b>Description</b> The server encountered an unexpected condition that prevented it from fulfilling the request.</p><hr class="line" /><h3>Apache Tomcat/10.1.13</h3></body></html>
